In [1]:
using Flux, MLDatasets, Statistics
using Flux: onehotbatch, onecold, logitcrossentropy
using Flux.Data: DataLoader
using MLDatasets: MNIST
using Base.Iterators: partition
using Printf, BSON
using Parameters: @with_kw
using CUDA
CUDA.allowscalar(false)

ArgumentError: ArgumentError: Package Parameters not found in current path.
- Run `import Pkg; Pkg.add("Parameters")` to install the Parameters package.

In [2]:
@with_kw mutable struct Args
    lr::Float64 = 3e-3
    epochs::Int = 100
    batch_size = 128
    savepath::String = "./"
 end

LoadError: LoadError: UndefVarError: @with_kw not defined
in expression starting at /home/tekim/vision_cctv/julia/prac1.ipynb:1

In [3]:
for i in 1:3
    print(Float32.(i))
end

1.02.03.0

In [45]:
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X)[1:end-1]..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[:,:,idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:9)
    return (X_batch, Y_batch)
 end

make_minibatch (generic function with 1 method)

In [46]:
train_imgs, train_labels = MNIST(split=:train)[:]

(features = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], targets = [5, 0, 4, 1, 9, 2, 1, 3, 1, 4  …  9, 2, 9, 5, 1, 8, 3, 5, 6, 8])

In [49]:
x = DataLoader(train_imgs)

ErrorException: type Array has no field nobs

In [37]:
function get_processed_data(args)
    # Load labels and images
    train_imgs, train_labels = MNIST(split=:train)[:]
    mb_idxs = partition(1:length(train_labels), args.batch_size)
    train_set = [make_minibatch(train_imgs, train_labels, i) for i in mb_idxs]
   
    # Prepare test set as one giant minibatch:
    test_imgs, test_labels = MNIST(split=:test)[:]
    test_set = make_minibatch(test_imgs, test_labels, 1:length(test_labels))
  
    return train_set, test_set
  
 end

get_processed_data (generic function with 1 method)

In [38]:
function build_model(args; imgsize = (28,28,1), nclasses = 10)
    cnn_output_size = Int.(floor.([imgsize[1]/8,imgsize[2]/8,32])) 
  
    return Chain(
    # First convolution, operating upon a 28x28 image
    Conv((3, 3), imgsize[3]=>16, pad=(1,1), relu),
    MaxPool((2,2)),
  
    # Second convolution, operating upon a 14x14 image
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),
  
    # Third convolution, operating upon a 7x7 image
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2,2)),
  
    # Reshape 3d array into a 2d one using `Flux.flatten`, at this point it should be (3, 3, 32, N)
    flatten,
    Dense(prod(cnn_output_size), 10))
 end

build_model (generic function with 1 method)

In [39]:
augment(x) = x .+ gpu(0.1f0*randn(eltype(x), size(x)))
anynan(x) = any(y -> any(isnan, y), x)
accuracy(x, y, model) = mean(onecold(cpu(model(x))) .== onecold(cpu(y)))


function train(; kws...)   
   args = Args(; kws...)
 
   @info("Loading data set")
   train_set, test_set = get_processed_data(args)
 
   # Define our model.  We will use a simple convolutional architecture with
   # three iterations of Conv -> ReLU -> MaxPool, followed by a final Dense layer.
   @info("Building model...")
   model = build_model(args)
 
   # Load model and datasets onto GPU, if enabled
   train_set = gpu.(train_set)
   test_set = gpu.(test_set)
   model = gpu(model)
  
   # Make sure our model is nicely precompiled before starting our training loop
   model(train_set[1][1])
 
   # `loss()` calculates the crossentropy loss between our prediction `y_hat`
   # (calculated from `model(x)`) and the ground truth `y`.  We augment the data
   # a bit, adding gaussian random noise to our image to make it more robust.
   function loss(x, y)   
       x̂ = augment(x)
       ŷ = model(x̂)
       return logitcrossentropy(ŷ, y)
   end
  
   # Train our model with the given training set using the ADAM optimizer and
   # printing out performance against the test set as we go.
   opt = ADAM(args.lr)
  
   @info("Beginning training loop...")
   best_acc = 0.0
   last_improvement = 0
   for epoch_idx in 1:args.epochs
       # Train for a single epoch
       Flux.train!(loss, params(model), train_set, opt)
      
       # Terminate on NaN
       if anynan(Flux.params(model))
           @error "NaN params"
           break
       end
  
       # Calculate accuracy:
       acc = accuracy(test_set..., model)
      
       @info(@sprintf("[%d]: Test accuracy: %.4f", epoch_idx, acc))
       # If our accuracy is good enough, quit out.
       if acc >= 0.999
           @info(" -> Early-exiting: We reached our target accuracy of 99.9%")
           break
       end
  
       # If this is the best accuracy we've seen so far, save the model out
       if acc >= best_acc
           @info(" -> New best accuracy! Saving model out to mnist_conv.bson")
           BSON.@save joinpath(args.savepath, "mnist_conv.bson") params=cpu.(params(model)) epoch_idx acc
           best_acc = acc
           last_improvement = epoch_idx
       end
  
       # If we haven't seen improvement in 5 epochs, drop our learning rate:
       if epoch_idx - last_improvement >= 5 && opt.eta > 1e-6
           opt.eta /= 10.0
           @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")
 
           # After dropping learning rate, give it a few epochs to improve
           last_improvement = epoch_idx
       end
  
       if epoch_idx - last_improvement >= 10
           @warn(" -> We're calling this converged.")
           break
       end
   end
end

train (generic function with 1 method)

In [40]:
function test(; kws...)
    args = Args(; kws...)
   
    # Loading the test data
    _,test_set = get_processed_data(args)
   
    # Re-constructing the model with random initial weights
    model = build_model(args)
   
    # Loading the saved parameters
    BSON.@load joinpath(args.savepath, "mnist_conv.bson") params
   
    # Loading parameters onto the model
    Flux.loadparams!(model, params)
   
    test_set = gpu.(test_set)
    model = gpu(model)
    @show accuracy(test_set...,model)
 end

test (generic function with 1 method)

In [41]:
train()

┌ Info: Loading data set
└ @ Main c:\Users\rlaxo\vision_cctv\julia\prac1.ipynb:9


┌ Info: Building model...
└ @ Main c:\Users\rlaxo\vision_cctv\julia\prac1.ipynb:14


UndefVarError: UndefVarError: flatten not defined